## Importing Libraries

In [16]:
import pandas as pd
import numpy as np
##
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
##
import time
import random
from datetime import datetime

#visulaization modules
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Historic Data

In [22]:
data_old = pd.read_csv('export_GDPR_fines_last.csv')

In [24]:
data_old.head()

,Unnamed: 0,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type,Source
0,0,ETid-987,IRELAND,2021-12-02,"60,000",Irish Teacher Council,"Art. 5 (1) GDPR, Art. 32 (1) GDPR, Art. 33 GDPR",Insufficient technical and organisational meas...,link
1,1,ETid-986,GREECE,2021-12-31,"30,000",INFO COMMUNICATION SERVICES,"Art. 13 GDPR, Art. 14 GDPR, Art. 11 Law 3471/2006",Insufficient fulfilment of information obligat...,link
2,2,ETid-985,GREECE,2021-12-31,"25,000",PLUS REAL ADVERTISEMENT,"Art. 13 GDPR, Art. 14 GDPR, Art. 11 Law 3471/2006",Insufficient fulfilment of information obligat...,link
3,3,ETid-984,POLAND,2021-12-09,"10,000",Warsaw University of Technology,"Art. 5 (1) f) GDPR, Art. 5 (2) GDPR, Art. 24 (...",Insufficient technical and organisational meas...,link link
4,4,ETid-983,SPAIN,2022-01-11,"9,000",EDUCANDO JUNTOS SL,"Art. 6 (1) GDPR, Art. 17 GDPR",Insufficient legal basis for data processing,link


## Scraping data

In [44]:
#getting last ID vs historic data
last_current_id = driver.find_element(By.XPATH, '//*[@id="penalties"]/tbody/tr[1]/td[2]/a').text
last_current_id = last_id.replace('ETid-', '')

last_historic_id = data_old.loc[0]['ID'].replace('ETid-', '')

delta = int(last_current_id) - int(last_historic_id)

pages_to_load = delta//100 + 1

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.enforcementtracker.com/")
driver.maximize_window()

In [17]:
select = Select(driver.find_element(By.XPATH, '//*[@id="penalties_length"]/label/select'))
# select by visible text
select.select_by_visible_text('100')

In [20]:
headers = []
for i in range(2,14):
    elem = driver.find_element(By.XPATH, '//*[@id="penalties"]/thead/tr[1]/th['+str(i)+']')
    if elem.text != "":
        headers.append(elem.text)

In [48]:
data = []
content = []

for page in range(1, pages_to_load + 1):

    print('Page ' + str(page) + ' / ' + str(pages_to_load) + ' en cours de chargement.')
    
    for i in range(1,101):
        
        for j in range(2,13):
            
            elem = driver.find_element(By.XPATH, '//*[@id="penalties"]/tbody/tr['+str(i)+']/td['+str(j)+']')
            
            if elem.text != '':
                content.append(elem.text)
    
        data.append(content)
        content = []
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
    ActionChains(driver).click(driver.find_element(By.XPATH, '//*[@id="penalties_paginate"]/span/a['+str(page+1)+']')).perform()        
    
    print("Page " + str(page) + ' / ' + str(pages_to_load) + " récupérée !")

    time.sleep(random.randint(2, 7))

Page 1 / 4 en cours de chargement.
Page 1 / 4 récupérée !
Page 2 / 4 en cours de chargement.
Page 2 / 4 récupérée !
Page 3 / 4 en cours de chargement.
Page 3 / 4 récupérée !
Page 4 / 4 en cours de chargement.
Page 4 / 4 récupérée !


## Loading Data

In [55]:
df_delta = pd.DataFrame(data, columns = ['ID','Country','Date','Amount','Controller_Processor','Quoted_Art','Type','Source'])

In [56]:
df_delta.shape

(400, 8)

In [ ]:
display(df_delta.head(5))
display(df_delta.info())

,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type,Source
0,ETid-1340,GERMANY,2021,Unknown,Private individual,Art. 5 (1) c) GDPR,Non-compliance with general data processing pr...,link
1,ETid-1339,GERMANY,2021,"Fine amount between EUR 100 and EUR 1,000",Private individual,Art. 6 GDPR,Insufficient legal basis for data processing,link
2,ETid-1338,GERMANY,2021,Unknown,Unknown,Art. 32 GDPR,Insufficient technical and organisational meas...,link
3,ETid-1337,GERMANY,2021,Unknown,Unknown,Art. 32 GDPR,Insufficient technical and organisational meas...,link
4,ETid-1336,GERMANY,2021,Unknown,Company,"Art. 25 GDPR, Art. 32 GDPR",Insufficient technical and organisational meas...,link


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    400 non-null    object
 1   Country               400 non-null    object
 2   Date                  400 non-null    object
 3   Amount                400 non-null    object
 4   Controller_Processor  400 non-null    object
 5   Quoted_Art            400 non-null    object
 6   Type                  400 non-null    object
 7   Source                400 non-null    object
dtypes: object(8)
memory usage: 25.1+ KB


None

In [57]:
data_old.drop(['Unnamed: 0', 'Source'], axis = 1)

,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type
0,ETid-987,IRELAND,2021-12-02,"60,000",Irish Teacher Council,"Art. 5 (1) GDPR, Art. 32 (1) GDPR, Art. 33 GDPR",Insufficient technical and organisational meas...
1,ETid-986,GREECE,2021-12-31,"30,000",INFO COMMUNICATION SERVICES,"Art. 13 GDPR, Art. 14 GDPR, Art. 11 Law 3471/2006",Insufficient fulfilment of information obligat...
2,ETid-985,GREECE,2021-12-31,"25,000",PLUS REAL ADVERTISEMENT,"Art. 13 GDPR, Art. 14 GDPR, Art. 11 Law 3471/2006",Insufficient fulfilment of information obligat...
3,ETid-984,POLAND,2021-12-09,"10,000",Warsaw University of Technology,"Art. 5 (1) f) GDPR, Art. 5 (2) GDPR, Art. 24 (...",Insufficient technical and organisational meas...
4,ETid-983,SPAIN,2022-01-11,"9,000",EDUCANDO JUNTOS SL,"Art. 6 (1) GDPR, Art. 17 GDPR",Insufficient legal basis for data processing
...,...,...,...,...,...,...,...
982,ETid-5,BELGIUM,2019-05-28,"2,000",Mayor,"Art. 5 (1) b) GDPR, Art. 6 GDPR",Insufficient legal basis for data processing
983,ETid-4,AUSTRIA,2018-12-20,"2,200",Private person,"Art. 5 (1) a) GDPR, Art. 5 (1) c) GDPR, Art. 6...",Insufficient legal basis for data processing
984,ETid-3,AUSTRIA,2018-09-27,300,Private car owner,"Art. 5 (1) a) GDPR, Art. 6 GDPR",Insufficient legal basis for data processing
985,ETid-2,AUSTRIA,2018,"1,800",Kebab restaurant,"Art. 5 GDPR, Art. 13 GDPR, Art. 14 GDPR",Insufficient legal basis for data processing


In [72]:
total_df = pd.concat([df_delta.drop('Source', axis = 1), data_old.drop(['Unnamed: 0', 'Source'], axis = 1)], ignore_index=True)

total_df = total_df.drop_duplicates(subset = 'ID')

total_df.head()

,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type
0,ETid-1340,GERMANY,2021,Unknown,Private individual,Art. 5 (1) c) GDPR,Non-compliance with general data processing pr...
1,ETid-1339,GERMANY,2021,"Fine amount between EUR 100 and EUR 1,000",Private individual,Art. 6 GDPR,Insufficient legal basis for data processing
2,ETid-1338,GERMANY,2021,Unknown,Unknown,Art. 32 GDPR,Insufficient technical and organisational meas...
3,ETid-1337,GERMANY,2021,Unknown,Unknown,Art. 32 GDPR,Insufficient technical and organisational meas...
4,ETid-1336,GERMANY,2021,Unknown,Company,"Art. 25 GDPR, Art. 32 GDPR",Insufficient technical and organisational meas...


In [73]:
#check unicité IDs
if total_df['ID'].value_counts().sort_values(ascending = False)[0] < 2:
    print('no duplicates !')
else:
    print('dedup to be done !')

no duplicates !


In [74]:
#export pour sécu
total_df.to_csv('export_GDPR_fines_last.csv')

## Data Cleaning

In [75]:
total_df

,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type
0,ETid-1340,GERMANY,2021,Unknown,Private individual,Art. 5 (1) c) GDPR,Non-compliance with general data processing pr...
1,ETid-1339,GERMANY,2021,"Fine amount between EUR 100 and EUR 1,000",Private individual,Art. 6 GDPR,Insufficient legal basis for data processing
2,ETid-1338,GERMANY,2021,Unknown,Unknown,Art. 32 GDPR,Insufficient technical and organisational meas...
3,ETid-1337,GERMANY,2021,Unknown,Unknown,Art. 32 GDPR,Insufficient technical and organisational meas...
4,ETid-1336,GERMANY,2021,Unknown,Company,"Art. 25 GDPR, Art. 32 GDPR",Insufficient technical and organisational meas...
...,...,...,...,...,...,...,...
1382,ETid-5,BELGIUM,2019-05-28,"2,000",Mayor,"Art. 5 (1) b) GDPR, Art. 6 GDPR",Insufficient legal basis for data processing
1383,ETid-4,AUSTRIA,2018-12-20,"2,200",Private person,"Art. 5 (1) a) GDPR, Art. 5 (1) c) GDPR, Art. 6...",Insufficient legal basis for data processing
1384,ETid-3,AUSTRIA,2018-09-27,300,Private car owner,"Art. 5 (1) a) GDPR, Art. 6 GDPR",Insufficient legal basis for data processing
1385,ETid-2,AUSTRIA,2018,"1,800",Kebab restaurant,"Art. 5 GDPR, Art. 13 GDPR, Art. 14 GDPR",Insufficient legal basis for data processing


In [80]:
def prepare_data(df):
    
    df['ID'] = df['ID'].str.replace('ETid-', '')

    df['Amount'] = df['Amount'].str.replace(' ','').str.replace(r'[a-zA-Z,-]','').replace(r'^\s*$', 0, regex=True)
    df['Amount'] = df['Amount'].fillna('0')
    df['Amount'] = df['Amount'].astype(int)
    
    df.loc[df['Amount'] == 0] = int(df['Amount'].median())
    
    df['Country'] = df['Country'].str.replace(' ', '_')
    
    df['Date'] = df['Date'].str.replace('Unknown','')
    df['Date'] = df['Date'].fillna(method = 'ffill')
    
    df[['DateYear','DateMonth','DateDay']] = df['Date'].str.split('-', expand = True)
    
    df['DateMonth'] = df['DateMonth'].fillna(6).astype('int')
    df['DateDay'] = df['DateDay'].fillna(15).astype('int')
    
    df = df.join(pd.get_dummies(df['DateYear'], prefix = 'Year'))
    
    df = df.join(pd.get_dummies(df['Country'], prefix='Country'))
    
    df = df.join(pd.get_dummies(df['Type'], prefix = 'Type'))
    
    GAFAM = ['FACEBOOK','AMAZON','GOOGLE','MICROSOFT', 'APPLE']  
    regstr = '|'.join(GAFAM)
    df['is_gafam'] = np.where(df['Controller_Processor'].str.upper().str.contains(regstr), 1, 0)

    
    #treatment of article column
    df['Quoted_Art'] = df['Quoted_Art'].str.replace(r'[a-zA-Z.§é]','').str.replace(' ','').str.replace(r"\(.*\)",'')
    df = df.join(pd.get_dummies(df['Quoted_Art'].str.split(',').apply(pd.Series).stack(), prefix = 'Art').sum(level=0))    
    
    df['VIOLATED_ARTICLES'] = df[[x for x in df.columns if "Art" in x]].sum(axis = 1)
    
    #ménage
    df = df.drop(['Unnamed: 0', 'Source', 'Date', 'ID', 'Country',
                  'Quoted_Art', 'Type', 'Art_3759/2004', 'Art_113471/2006',
                    'Art_','DateYear', 'Controller_Processor'], axis = 1)
    
    
    #df['Amount'] = np.log(df['Amount'])
    
    return df

In [81]:
data = prepare_data(total_df)

C:\Users\Valentin\AppData\Local\Temp\ipykernel_14188\1006689672.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Amount'] = df['Amount'].str.replace(' ','').str.replace(r'[a-zA-Z,-]','').replace(r'^\s*$', 0, regex=True)
C:\Users\Valentin\AppData\Local\Temp\ipykernel_14188\1006689672.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Quoted_Art'] = df['Quoted_Art'].str.replace(r'[a-zA-Z.§é]','').str.replace(' ','').str.replace(r"\(.*\)",'')
C:\Users\Valentin\AppData\Local\Temp\ipykernel_14188\1006689672.py:34: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df = df.join(pd.get_dummies(df['Quoted_Art'].str.split(',').apply(pd.Series).stack(), prefix = 'Art').sum(level=0))
C:\Users\Valentin\AppData\Local\Temp\ipykernel_1

KeyError: "['Unnamed: 0', 'Source'] not found in axis"

In [ ]:
def assess_model(df, model):
    
    target = df['Amount']
    features = df.drop(['Amount'], axis = 1)
    
    X_train, X_test, y_train, y_test = train_test_split(features, target,train_size = 0.8, random_state = 42)
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    score = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    
    scores = cross_val_score(model, X_train, y_train, scoring='r2', cv=5)
    
    print(model)
    print('MSE : '+ str(score))
    print('R2 : '+str(r2))
    print('CV scores : '+ str(scores))    
    return

In [ ]:
.